In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
cap = cv2.VideoCapture('linusi.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
